<a href="https://colab.research.google.com/github/SuvarnaDalin/Analytics-Projects/blob/master/Collaborative_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
!git clone https://github.com/SuvarnaDalin/Data-for-Analysis.git

Cloning into 'Data-for-Analysis'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 18 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (18/18), done.


In [ ]:
!cd Data-for-Analysis/

In [ ]:
!ls Data-for-Analysis/

Iris.csv  posts.csv  README.md	SampleSuperstore.csv  users.csv  views.csv


In [ ]:
posts_data = pd.read_csv('Data-for-Analysis/posts.csv')
users_data = pd.read_csv('Data-for-Analysis/users.csv')
views_data = pd.read_csv('Data-for-Analysis/views.csv')

Recommendation System, based on:
# 2. Collaborative Filtering

Preprocessing the data

In [ ]:
df1 = views_data[['user_id', 'post_id']]
df1.columns = ['_id' if x=='user_id' else x for x in df1.columns]
df2 = users_data[['_id', 'name']]
df3 = pd.merge(df2, df1)
df3.head()

,_id,name,post_id
0,5d60098a653a331687083238,Nivesh Singh Chauhan,5ed3820f76027d35905ccac8
1,5d60098a653a331687083238,Nivesh Singh Chauhan,5ed1ff0276027d35905cc60d
2,5d60098a653a331687083238,Nivesh Singh Chauhan,5ecf96e876027d35905cbf46
3,5d60098a653a331687083238,Nivesh Singh Chauhan,5ecfa0ca76027d35905cbf57
4,5d60098a653a331687083238,Nivesh Singh Chauhan,5ed0e20776027d35905cc2fe


Mapping user name to post_id to obtain the number of views made by a user.

In [ ]:
views_df = df3
views_crossed = pd.crosstab(views_df['name'], views_df['post_id']).fillna(0)
views_crossed['5eb2c11210426255a7aaa052'].sum()

14

Creating a sparse matrix

In [ ]:
from scipy.sparse import csr_matrix
views_mat = csr_matrix(views_crossed.values)

Model Fitting using KNN

In [ ]:
from sklearn.neighbors import NearestNeighbors
#make an object for the NearestNeighbors Class.
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)# fit the dataset

model_knn.fit(views_mat)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=-1, n_neighbors=20, p=2,
                 radius=1.0)

### Result: Recommend posts for the given user

In [ ]:
pip install fuzzywuzzy

In [ ]:
from fuzzywuzzy import process

def recommender(user_name, data,model, n_recommendations):
    model.fit(data)
    idx=process.extractOne(user_name, views_df['post_id'], score_cutoff=0)[2]
    print('Name Selected: ', user_name, 'Index: ',idx)
    print('Searching for post recommendations.....')
    distances, indices=model.kneighbors(data[idx], n_neighbors=n_recommendations)
    for i in indices:
        print(views_df['post_id'][i].where(i!=idx))

/usr/local/lib/python3.6/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
# Give any name from the given snippet to check the results
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
array(['Nivesh Singh Chauhan', 'Gaurav Sharma', 'Akshay Mishra',
       'Saksham Mathur', 'Varun Chowhan', 'Boshi Kaviya',
       'Nischaya Sharma', 'thesocialcomment', 'MAAZ AHMAD RABBANI',
       'siddharth saxena', 'BHASKAR MISHRA', 'hritik chaturvedi',
       'Aryan Malakar', 'Parvez Khan', 'Angelina Christi',
       'Niriksha Sharma', 'Nimish Sharma', 'Rashi Jain',
       'Nachiket Khatri', 'Ojasvi Arya', 'Sneha Shekhawat',
       'Nikhil Sharma', 'Utkarsh Saini', 'Arunav Moitra',
       ...........................................................
       'Parth Praveen Deokar', 'RAGHUNATH DHANDAPANI', 'Rajesha CT',
       'Saumik Satapathy', 'Manish Nehra', 'Mohd Junaid Mansuri',
       'Mukesh', 'Neha', 'Saumya Singh', 'Arman Lalani', 'BOBIL SINGH',
       'RUSHIKESH CHAUDHARI', 'Steve', 'Himanshu Singh', 'Parth Vijay',
       'Abhishek Mishra', 'gauri raskar'], dtype=object)
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

recommender('Saumya Singh', views_mat, model_knn,20)


Name Selected:  Saumya Singh Index:  31
Searching for post recommendations.....
31                         NaN
75    5e9806f5a3258347b42f2b61
66    5eabfaf510426255a7aa9da0
62    5eb1ac4510426255a7aaa024
64    5eac350110426255a7aa9e92
88    5ea3236810426255a7aa9ac8
76    5ea7cd9610426255a7aa9bd2
77    5e9eececa3258347b42f2e11
78    5ea323a010426255a7aa9aca
79    5ea8618a10426255a7aa9c0d
80    5eaaffe110426255a7aa9c5b
81    5ea1603a10426255a7aa9a3e
82    5e9db770a3258347b42f2dcf
83    5ea11fbd10426255a7aa9a38
84    5ea319ee10426255a7aa9aa4
85    5ea1ef0710426255a7aa9a7e
86    5ea31ee710426255a7aa9ab3
87    5ea3227010426255a7aa9ac1
63    5d62abaa65218653a132c956
68    5ea30f6010426255a7aa9a9c
Name: post_id, dtype: object


### Result: Recommend similar posts for the given post

In [ ]:
def recommender(posts_id, data,model, n_recommendations):
    model.fit(data)
    idx=process.extractOne(posts_id, views_df['post_id'], score_cutoff=0)[2]
    print('Post Selected: ',views_df['post_id'][idx], 'Index: ',idx)
    print('Searching for similar post recommendations.....')
    distances, indices=model.kneighbors(data[idx], n_neighbors=n_recommendations)
    for i in indices:
        print(views_df['post_id'][i].where(i!=idx))

In [ ]:
# Give any post_id from the given snippet to check the results
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
array(['5ed3820f76027d35905ccac8', '5ed1ff0276027d35905cc60d', '5ecf96e876027d35905cbf46', '5ecfa0ca76027d35905cbf57',
       '5ed0e20776027d35905cc2fe', '5ecfffb976027d35905cc0e9', '5ecfafcc76027d35905cbfb0', '5ed09ad076027d35905cc15f',
       '5e90208ca3258347b42f2730', '5e9028cea3258347b42f2736', '5e91a2dda3258347b42f276d', '5e895ceca3258347b42f25b6', 
       '5e8c993da3258347b42f264d', '5e4edd87f5561b1994c8e497', '5e4ba8b2f5561b1994c8e39f', '5e84657da3258347b42f2452',
       '5db2b91a99cbb90e4339c749', '5e81bc41a3258347b42f220a', '5e81bf73a3258347b42f2257', '5e81bf2fa3258347b42f2244',
       '5e81bfe5a3258347b42f226a', '5e81c06ba3258347b42f227d', '5e81c0cba3258347b42f2290', '5e81c101a3258347b42f22a3',
       '5e81c15aa3258347b42f22b6', '5e81c1e6a3258347b42f22dd', '5e78ae4ccfc8b713f5ac7cde', '5e7c78fdcfc8b713f5ac7daa',
       '5d80e7c16c53455f896e6014', '5eca7601ec493f4a26558a62', '5ed141aa76027d35905cc4c9', '5ed175f176027d35905cc554',
       '5ed0f0c576027d35905cc3c7', '5ed0e65976027d35905cc356',
              ...........................................................
       '5ed237d676027d35905cc6bd', '5ed23c1876027d35905cc761', '5ed23c8676027d35905cc780', '5ed23cf876027d35905cc790',
       '5ed23e4d76027d35905cc7b8', '5ed2378276027d35905cc6b5', '5de179d80eb5e25a8a07f079', '5ed175f476027d35905cc562',
       '5ed0007a76027d35905cc0ea', '5ed23d4276027d35905cc798', '5ed4cbadbd514d602c1531a6']     
       
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

recommender('5ec57914f2781131cc7e51c8', views_mat, model_knn,20)

Post Selected:  5ec57914f2781131cc7e51c8 Index:  26
Searching for similar post recommendations.....
26                          NaN
96     5e964bcea3258347b42f2a96
90     5e9f3e1aa3258347b42f2e39
68     5ea30f6010426255a7aa9a9c
86     5ea31ee710426255a7aa9ab3
64     5eac350110426255a7aa9e92
22     5eca8fceeaff6b0c3a58a3c0
40     5ec2215374f7660d73aa1011
115    5e9754efa3258347b42f2ae8
108    5e9a7e73a3258347b42f2c24
49     5eb1407c10426255a7aa9fdb
57     5eaf8b9310426255a7aa9f7e
67     5eabeec210426255a7aa9cd2
79     5ea8618a10426255a7aa9c0d
80     5eaaffe110426255a7aa9c5b
82     5e9db770a3258347b42f2dcf
83     5ea11fbd10426255a7aa9a38
84     5ea319ee10426255a7aa9aa4
81     5ea1603a10426255a7aa9a3e
73     5eab962e10426255a7aa9cb8
Name: post_id, dtype: object
